In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import re
import os
from tqdm import tqdm
import random

## Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/ITMO/RW'23")
DATA_DIR = BASE_DIR.joinpath("data")

FILE_NAME = "data_topic_name_distrib.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [ ]:
data = pd.read_csv(FILE)
data.head(10)

,INPUT:wordset,OUTPUT:quality,OUTPUT:topic_name,topic_name_num
0,able amaze fill stop glad daily fantastic move...,good,feelings,1
1,absolute mission church nasa mars earth truth ...,good,astrology,1
2,absolute mission church nasa mars earth truth ...,good,nasa,1
3,absolute mission church nasa mars earth truth ...,good,space,1
4,absolute mission church nasa mars earth truth ...,good,space mission,1
5,absolutely fast white rice service noodle terr...,good,customer experience,1
6,absolutely fast white super service customer r...,good,express expedition,1
7,absolutely fast white super service customer r...,good,online shopping,2
8,absolutely fast white super service customer r...,good,review,1
9,absolutely grain there's process terrible expe...,good,comment,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5766 entries, 0 to 5765
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   INPUT:wordset      5766 non-null   object
 1   OUTPUT:quality     5766 non-null   object
 2   OUTPUT:topic_name  5766 non-null   object
 3   topic_name_num     5766 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 180.3+ KB


In [ ]:
data_text = list(set(data["INPUT:wordset"]))

##Mistral-7B

In [ ]:
!nvidia-smi

Sun Jan 14 20:33:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

##Inference

In [ ]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [ ]:
def extract_substring(input_string):
    # pattern = r'\[/INST\](.*?)</s>'
    # match = re.search(pattern, input_string)
    # if match:
    #     return match.group(1)
    # else:
    #     return None
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [ ]:
def few_shot_pipeline(instruction, sample, model, device='cpu', temp=0.1, n_token=100):
    # Construct prompt
    # prompt = f"""[INST] {instruction}\n
    # Text: {sample}
    # Topic:[/INST]"""
    prompt = f"""<s> [INST] {instruction} {sample} [/INST]"""
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    result = extract_substring(answer[0])
    return result.replace("\n", " ")


####Type I

In [ ]:
prompt = """[INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])


[INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: technology</s>


In [ ]:
prompt = """[INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])


[INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] 1. space
2. news
3. [no topic found in the given text]</s>


In [ ]:
# type1_1
prompt = """[INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic:[/INST] Space exploration</s>


In [ ]:
prompt = """[INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic:[/INST] Topic: Anonymous, Public, Private, News, Crime, Server, Religion, Carry, Pretty, Bill, Cview, Internet, Paul, Hold, Thought.</s>


In [ ]:
prompt = """[INST] Name the common topic with one or few words. Please, make sure you to only return the label and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] Name the common topic with one or few words. Please, make sure you to only return the label and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic:[/INST] space, mission, church, nasa, mars, earth, truth, lunar, shuttle, doug, telescope, planetary, libxmu, launch</s>


####Type II

In [ ]:
prompt = """[INST] Analyze the list of words and name the common topic with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

[INST] Analyze the list of words and name the common topic with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] 1. "absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch" - space exploration
2. "bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry" - breakfast cereals
3. "apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason" - technology and computing</s>


In [ ]:
prompt = """[INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

[INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space

Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: computer and technology</s>


In [ ]:
prompt = """[INST] Analyze the list of words and name the common topic with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

[INST] Analyze the list of words and name the common topic with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST] Technology, Computers, Software, Music, Microsoft, Processor, Speed, Clock, Chicago, Jason.</s>


In [ ]:
# type2_1
prompt = """[INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

[INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST] Technology</s>


####Type III

In [ ]:
prompt = """[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: Technology</s>


In [ ]:
prompt = """[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: technology</s>


In [ ]:
# type 3_1
prompt = """[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST] The common topic in the given list of words is "news" and "crime".</s>


In [ ]:
prompt = """[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST] Crime, Religion, Internet, News, Politics</s>


In [ ]:
prompt = """[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the label and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the label and nothing more.
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: [/INST] crime, news, religion, internet, bill, cview, paul, hold, thought</s>


####Type IV

In [ ]:
prompt = """[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: technology</s>


In [ ]:
prompt = """[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the label and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the label and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] Label: technology

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: techno

In [ ]:
prompt = """[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] * apple: fruit
* mouse: computer mouse
* robert: name
* fast: speed
* processor: computer component
* speed: computer performance
* move: action
* slow: speed
* clock: timekeeping device
* week: time period
* pretty: appearance
* microsoft: company
* music: genre of sound
* chicago: city
* jason: name</s>


In [ ]:
prompt = """[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the label and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] apple: fruit
mouse: computer mouse
robert: name
fast: speed
processor: computer component
speed: performance
move: action
slow: pace
clock: timekeeping device
week: time unit
pretty: appearance
microsoft: company
music: genre
chicago: city
jason: name</s>


In [ ]:
prompt = """[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.3, n_token=100)

print(answer[0])

[INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] apple, mouse, robert, fast, processor, speed, move, slow, clock, week, pretty, microsoft, music, chicago, jason</s>


In [ ]:
prompt = """[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason [/INST] The main theme that combines all the words from the list is "technology".</s>


In [ ]:
prompt = """[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch [/INST] The main theme that combines all the words from the list is "Space".</s>


In [ ]:
prompt = """[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought [/INST] The main theme that combines all the words from the list is "Privacy".</s>


In [ ]:
prompt = """[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: greek turkish turkey cross armenia henrik turk greece plane azeri link cyprus allocation minority conflict [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0])

[INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: greek turkish turkey cross armenia henrik turk greece plane azeri link cyprus allocation minority conflict [/INST] The main theme that combines all the words from the list is "ethnicity."</s>


##Type I -> All dataset

In [ ]:
INSTRUCTION = """Name the common topic with one or few words."""

SAMPLE = data_text[0]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda")

res

' Peace Proposal'

In [ ]:
random.seed(42)
random.shuffle(data_text)

In [ ]:
data_text[:10]

['policy peace israeli arab israel proposal tyre center research border elia country nmsu conflict subject',
 'drinking mild espresso aftertaste brewing latte sip weird enjoy way guest might machine still appeal',
 'local piece grocery sell deal carry market glad online break hop surprised unfortunately whole flour',
 "value teeth difficult msg pea bone spent rawh jasmine glutamate what's monosodium europe inexpensive potential",
 'bill fire koresh clinton scott child batf compound stratus waco cult tear agent burn today',
 'way drinking mild aftertaste espresso weird sip latte either oppose mellow appeal richer overprice wonderfully',
 'nasa launch orbit satellite mission moon earth shuttle rocket lunar flight spacecraft probe mars solar',
 'price phone bike sell email company offer cost sale send technology internet distribution money looking',
 'printer radio signal print design output input quality audio engineering level build circuit digital electronics',
 'scsi controller fast i

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type1_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(data_text))):
    # for i in tqdm(range(10)):
        SAMPLE = data_text[i]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.2,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [58:53<00:00,  1.50s/it]


##Type II Entire Dataset

In [ ]:
INSTRUCTION = """Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more."""

SAMPLE = data_text[102]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda")

res

' space'

In [ ]:
random.seed(42)
random.shuffle(data_text)

In [ ]:
data_text[:10]

['care marriage insurance couple child poor mark public abortion money coverage live free health rent',
 'offer copy sale asking condition manual email title sell price excellent brand original shipping send',
 'player baseball season period brave league ball smith score hitter pitch pitcher boston pitching dave',
 'window display screen color manager server application create pixel visual client colors size colormap pixmap',
 'item deal shop anymore supply donut sampler many lavazza buck recieved disappoint cornbread dunkin kill',
 'israel israeli palestinian west bank suck joke arab bias medium territory stup palestine report cand',
 'mark care couple insurance marriage married coverage rent abortion poor speech wilson marry clinic free',
 'oil popcorn spice everything pop season olive chai cooking christmas garlic fish movie worst mustard',
 'monitor speed fast card mode cards hardware machine memory upgrade clock processor slow performance cache',
 'israel pres israeli encryption p

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type2_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)

res_df = pd.read_csv(RES_FILE, names=[1,2], sep=";")

In [ ]:
res_df

,1,2
0,player period season baseball boston pittsburg...,baseball
1,science theory scientific knowledge process ro...,"science, theory, knowledge, process, experime..."
2,turkish muslim greek turk turkey islam populat...,"The common topic in the list of words is ""Isl..."
3,calorie protein low fat coconut diet fiber sod...,"The common topic in this list of words is ""di..."
4,turkish muslim turk greek turkey armenia genoc...,The common topic in the given list of words i...
...,...,...
1870,nasa science research earth center anonymous o...,"NASA, science, research, earth, center, anony..."
1871,science theory scientific knowledge process ro...,"science, theory, knowledge, process, experime..."
1872,scsi controller interface disk transfer bios f...,"scsi controller, interface, disk transfer, bi..."
1873,quite expect bread wish recipe thing melt bake...,Baking


In [ ]:
words_lst = res_df[1]

In [ ]:
new_words_lst = [words for words in data_text if words not in list(words_lst)]

In [ ]:
print(len(data_text))
print(len(new_words_lst) + len(words_lst))

2363
2363


In [ ]:
len(new_words_lst)

488

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type2_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# # Remove old file
# if os.path.exists(RES_FILE):
#     os.remove(RES_FILE)
for i in tqdm(range(len(new_words_lst))):
# for i in tqdm(range(10)):
    SAMPLE = data_text[i]
    res_model = few_shot_pipeline(
        INSTRUCTION,
        SAMPLE,
        model,
        device="cuda",
        temp=0.3,
        n_token=100
    )
    with open(RES_FILE, 'a') as file:
      file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 488/488 [23:41<00:00,  2.91s/it]


##Type III Entire Dataset

In [ ]:
INSTRUCTION = """You will receive a list of words and you should analyze it and name the common topic with one or few words."""

SAMPLE = data_text[180]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda")

res

' The common topic in the given text is "diet" and "healthy eating."'

In [ ]:
random.seed(42)
random.shuffle(data_text)

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type3_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(data_text))):
    # for i in tqdm(range(10)):
        SAMPLE = data_text[i]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.3,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [1:41:28<00:00,  2.58s/it]


### NEW

In [ ]:
INSTRUCTION = """You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word:"""

SAMPLE = data_text[180]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda")

res

<s> [INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: health medical national cancer increase among report child research center april newsletter page fnal volume [/INST]


' The main theme that combines all the words from the list is "health."</s>'

In [ ]:
random.seed(42)
random.shuffle(data_text)

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type_new.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
for i in tqdm(range(len(data_text))):
# for i in tqdm(range(20)):
    torch.cuda.empty_cache()
    SAMPLE = data_text[i]
    res_model = few_shot_pipeline(
        INSTRUCTION,
        SAMPLE,
        model,
        device="cuda",
        temp=0.3,
        n_token=100
    )
    # print(res_model)
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [1:23:16<00:00,  2.11s/it]
